In [ ]:
# cd ~/twinews-logs ; jupython -o nohup-detokenize.out --venv st-venv ~/notebooks/twinews/dataset/detokenize.ipynb

In [1]:
isNotebook = '__file__' not in locals()

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.tokenizer import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [3]:
logger = Logger(tmpDir('logs') + "/twinews-detokenize.log") if isNotebook else Logger("detokenize.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [4]:
TEST = isNotebook

In [5]:
newsCollection = getNewsCollection(user='hayj')

twinews news (version 1.0) initialised.


In [6]:
ids = list(newsCollection.distinct("_id"))
tt.tic("Got ids")

--> tic: 5.17s | message: Got ids


5.17

In [7]:
ids = chunks(ids, 10 if TEST else 1000)
if TEST:
    ids = ids[:3]

In [8]:
bp(ids, logger)

[
  [ 5e70a0c6137cc1977b8b3a53, 5e70a0c6137cc1977b8b3a54, ..., 5e70a0c6137cc1977b8b3a5c, 5e70a0c6137cc1977b8b3a5d ],
  [ 5e70a0c6137cc1977b8b3a5e, 5e70a0c6137cc1977b8b3a5f, ..., 5e70a0c6137cc1977b8b3a66, 5e70a0c6137cc1977b8b3a68 ],
  [ 5e70a0c6137cc1977b8b3a6a, 5e70a0c6137cc1977b8b3a6b, ..., 5e70a0c6137cc1977b8b3a72, 5e70a0c6137cc1977b8b3a73 ]
]


In [9]:
def genFunct(containers, *args, **kwargs):
    if not isinstance(containers[0], list):
        containers = [containers]
    newsCollection = getNewsCollection()
    for chunk in containers:
        for currentId in chunk:
            row = newsCollection.findOne({'_id': currentId}, projection={'sentences': True, 'detokSentences': True})
            if not dictContains(row, 'detokSentences'):
                detokSentences = row['sentences']
                for i in range(len(detokSentences)):
                    detokSentences[i] = detokenize(detokSentences[i])
                detokText = " ".join(detokSentences)
                yield (currentId, detokSentences, detokText)

In [10]:
mli = MLIterator(ids, genFunct, logger=logger, parallelProcesses=cpuCount(), printRatio=0.0001, queuesMaxSize=50000)

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
3 containers to process.


In [11]:
i = 0
for (currentId, detokSentences, detokText) in mli:
    if TEST:
        bp(currentId, logger)
        bp(detokSentences, logger)
        log(detokText, logger)
    newsCollection.updateOne({'_id': currentId}, {'$set': {'detokSentences': detokSentences, 'detokText': detokText}})
    if TEST:
        log(i, logger)
    i += 1

twinews news (version 1.0) initialised.
twinews news (version 1.0) initialised.
twinews news (version 1.0) initialised.


5e70a0c6137cc1977b8b3a53
[
  Carolina Panthers surprises Triad couple who wedded in hospice chapel on Valentine Day The Carolina ,
  Lexington couple Kenneth and Charlene Smith were married Wednesday in the chapel at Hospice of David,
  ...,
  Kenneth and Charlene got married on Valentine Day in their Panthers gear.,
  @ThomasDavisSDTM delivered wedding gifts with a note from Coach KeepPounding Carolina Panthers (Febr
]
Carolina Panthers surprises Triad couple who wedded in hospice chapel on Valentine Day The Carolina Panthers made a surprise visit to the Triad to visit the couple who chose Valentine Day to say their "I do" at a local hospice house while wearing gear from their favorite football team. Lexington couple Kenneth and Charlene Smith were married Wednesday in the chapel at Hospice of Davidson County Kenneth is a patient at the hospice house. The couple has been together for __int_2__ years. The bride and groom chose to forego the traditional fancy tux and poofy white dresses

WKBW - Orchard Park - For most Western new Yorkers, nothing is more important than their faith and their family, but for many of those same people, the Buffalo Bills are right behind them. Those fans are understandably excited about the Bills first playoff appearance in __int_2__ seasons, but when some fans looked at the playoff schedule, they saw a big conflict. Just over __int_2__ teenagers from Nativity of Our Lord Parish in Orchard Park were scheduled to be confirmed Sunday at in the afternoon, just ten minutes after the scheduled kickoff in Jacksonville, but where some fans saw a problem, Bishop Malone saw a way to help. "The bishop makes it clear that confirmation is much more important than sports. Says George Richert, director of Communications for the Buffalo diocese. "But confirmation should also be a happy day and a memorable day in the life of any Catholic, and we would want that distraction. Seeing an opening in his schedule, the Bishop agreed to celebrate two confirmation

6
5e70a0c6137cc1977b8b3a6c
[
  When did Republicans lose their damn minds?,
  We need to 'splain you yesterday crazy Gowdy Nunes Uranium One fuckery, but honestly we a little wor,
  ...,
  Also, pay for the Wonkserver!,
  Mother Jones Reuters Cummings Schiff Letter Medium
]
When did Republicans lose their damn minds? We need to 'splain you yesterday crazy Gowdy Nunes Uranium One fuckery, but honestly we a little worried you going to start backing away from us whispering, Poor Wonkette! Looks like the strain of covering the Trump Administration finally got to them. And it has! But stick with us, because there is a vast rightwing, Hillary - hatin 'conspiracy going on with House Republicans! And we are FOR SHOCKED. (No, for real this time! This stuff is NUTS. Gowdy and Nunes have literally been hiding a witness that the DOJ told them was lying, so that they could go on Fox and scream, HILLARY! BRIBES! URANIUM ONE! So weird that Trey Gowdy has suddenly decided to retire, right? But we gett

7
5e70a0c6137cc1977b8b3a61
[
  Do feel alone on Valentine Day.,
  Valentine Day is the "official" day of love and romance.,
  ...,
  This article was originally published at Kelly Ann Garnett.,
  Reprinted with permission from the author.
]
Do feel alone on Valentine Day. Valentine Day is the "official" day of love and romance. It the day that women are supposed to be adored by their man with gifts of flowers, chocolate, and a prix fix menu at a fancy (meaning expensive) restaurant. Unless you single on Valentine Day. And then you get nothing. Hello, Singles Awareness Day. The greatest challenge with the holidays, especially Valentine Day, is the expectations that are placed on these days, especially with regards to romance. Many single people feel like the marketing during this time is almost mocking the fact that you not in a relationship hence the nickname, Singles Awareness Day. But just because you currently sans a man, who says you ca still be adored with love and romance? That r

...

22
5e70a0c6137cc1977b8b3a58
[
  You must enter the characters with black color that stand out from the other characters Whether it o,
  This month, it biting into Tide Pods.,
  ...,
  They need to "recruit the most influential Youtuber to talk about how this is negative, she said.,
  "If that sets the tone, then everything else will fall.
]
You must enter the characters with black color that stand out from the other characters Whether it out - of - control drinking games or the latest social media challenge, teens always seem to be into something that makes absolutely no sense at all. This month, it biting into Tide Pods. According to the American Association of Poison Control Centers, a growing number of teenagers are choosing to chomp into single - load laundry detergent packets for likes, clicks or views on social media. This week, the country poison control centers issued a "high alert, reporting that, in the first two weeks of __int_4__ they had handled __int_2__ such cases among 

23
5e70a0c6137cc1977b8b3a59
[
  CHARLESTON, WV (This week, bills will drop in the West Virginia House and Senate to add sexual orien,
  LGBT advocates have been pushing for the change since a old gay man in Marion County was beaten to d,
  ...,
  But not everyone agrees that this bill is best for West Virginia.,
  Share your thoughts by commenting below.
]
CHARLESTON, WV (This week, bills will drop in the West Virginia House and Senate to add sexual orientation and gender identity to the state hate crime law. LGBT advocates have been pushing for the change since a old gay man in Marion County was beaten to death in __int_4__ But supporters say they believe this could be the year West Virginia changes it law. According to data released just two months ago from the FBI, in __int_4__ more than people were attacked because of their sexual orientation, and __int_3__ for their gender identity. With hate crimes up __int_2__ overall in __int_4__ human rights advocates say now is the time to pr

25
 66% [=============       ] (7.255s left)
5e70a0c6137cc1977b8b3a5a
[
  The labour movement should demand that not just Carillion, but all the big public service contractor,
  Their workers - including agency staff and formally - self - employed contractors - should be brough,
  ...,
  It not just Carillion.,
  Bring all the big contracted - out operations back in - house!
]
The labour movement should demand that not just Carillion, but all the big public service contractors, Serco, Capita, should be nationalised with minimum compensation, and their public contracts brought back in - house. Their workers - including agency staff and formally - self - employed contractors - should be brought into union - negotiated public - sector pay and conditions, and the services should be run under the democratic control of the workers and elected public authorities. We must demand damage limitation for Carillion workers. Carillion public - sector contracts should be taken in - house, and its pri

Of Course Florida School Shooter Was A Girl - Hating White Supremacist. Of Course I Am Tired Of Writing This Article. Add to Flipboard Magazine. A few hours after the final body count of the latest school shooting was tallied, it came out: Nikolas Cruz, the shooter, had been known to "harass" women, known to abuse his ex - girlfriend, and was kicked out of school for getting into a fight with his ex - girlfriend. Before long he was being cheered on by the sociopaths over on __netloc__ K board, and Reddit new incel community r braincels. This is what happens now. I wrote this article when Elliot Rodger shot up Isla Vista, and I wrote it again when Christopher Harper - Mercer shot up Umpqua Community College. Twice. Each time it happens the same way: Some man with clear issues with women goes on a murder spree, and each time he is cheered as a hero on misogynistic forums. We know now that Cruz was a Trump supporter who often wore a MAGA hat, that he was known to say horrible things about

27
5e70a0c6137cc1977b8b3a5c
[
  The NCAA has opened an investigation into Michigan State University handling of sexual abuse allegat,
  Nassar has pleaded guilty to seven counts of criminal sexual conduct, and admitted to sexually assau,
  ...,
  He faces up to __int_3__ years in prison, and has already been sentenced to __int_2__ years in priso,
  The - CNN - Wire __int_4__ Cable News Network, Inc., a Time Warner Company.
]
The NCAA has opened an investigation into Michigan State University handling of sexual abuse allegations against sports doctor Larry Nassar. Nassar has pleaded guilty to seven counts of criminal sexual conduct, and admitted to sexually assaulting young girls when he treated campus athletes and members of the USA Gymnastics. Prominent Olympians including Simone Biles, Aly Raisman, Gabby Douglas and McKayla Maroney, have said they were abused by Nassar. In a statement, the NCAA said it looking into whether the university violated any rules. "The NCAA has sent a lette

In [12]:
tt.toc()

--> toc total duration: 25.57s


25.57